In [1]:
from __future__ import print_function 
from keras.models import Model
from keras.layers import Input,LSTM,Dense
import numpy as np

In [2]:
batch_s=64
epochs=100
latent_dimension=256
number_samples=10000  
data_path='./fra.txt'

In [3]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open (data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines[:min(number_samples,len(lines)-1)]:
    input_text,target_text,_=line.split('\t')
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
len(input_texts)

10000

In [5]:
len(target_texts)

10000

In [6]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

In [7]:
max_encoder_seq_length=max([len(text) for text in input_texts])
max_decoder_seq_length=max([len(text) for text in target_texts])

In [8]:
max_decoder_seq_length

59

In [9]:
max_encoder_seq_length

14

In [10]:
num_encoder_tokens

70

In [11]:
num_decoder_tokens

91

In [12]:
input_token_index=dict(
    [(char,i) for i,char in enumerate(input_characters)]
)

In [13]:
num_encoder_tokens

70

In [14]:
len(input_characters)

70

In [15]:
target_token_index=dict(
    [(char,i) for i,char in enumerate(target_characters)]
)

In [16]:
len(input_texts)

10000

In [17]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),
    dtype='float32'
)

In [18]:
encoder_input_data.shape

(10000, 14, 70)

In [19]:
decoder_input_data=np.zeros(
    (len(target_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32'
)

In [20]:
decoder_target_data=np.zeros(
    (len(target_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32'
)

In [22]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1.
    encoder_input_data[i,t+1:,input_token_index[' ']]=1.
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1.
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1.
    decoder_input_data[i,t+1:,target_token_index[' ']]=1.
    decoder_target_data[i,t:,target_token_index[' ']]=1.
            

In [24]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dimension,return_state=True)

In [25]:
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

In [26]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dimension,return_sequences=True,return_state=True)
decoder_output,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_output)

In [27]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [28]:
model.compile(optimizer='adam',loss='categorical_crossentropy',
             metrics=['accuracy'])

In [29]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
batch_size=batch_s,
epochs=10,
validation_split=0.2)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 103ms/step - accuracy: 0.7037 - loss: 1.8249 - val_accuracy: 0.7032 - val_loss: 1.0320
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 97ms/step - accuracy: 0.7555 - loss: 0.8867 - val_accuracy: 0.7638 - val_loss: 0.8703
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.7967 - loss: 0.7476 - val_accuracy: 0.7976 - val_loss: 0.7354
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 99ms/step - accuracy: 0.8222 - loss: 0.6402 - val_accuracy: 0.8119 - val_loss: 0.6685
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.8358 - loss: 0.5760 - val_accuracy: 0.8207 - val_loss: 0.6219
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 97ms/step - accuracy: 0.8454 - loss: 0.5369 - val_accuracy: 0.8259 - val_loss: 0.5946
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.8498 - loss: 0.5166 - val_accuracy: 0.8296 - val_loss: 0.5772
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - accuracy: 0.8572 - loss: 0.488

In [34]:
encoder_model=Model(encoder_inputs,encoder_states)
decoder_state_inputs_h=Input(shape=(latent_dimension,))
decoder_state_inputs_c=Input(shape=(latent_dimension,))
decoder_state_inputs=[decoder_state_inputs_h,decoder_state_inputs_c]
decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_state_inputs)
decoder_state=[state_h,state_c]

decoder_model=Model(
    [decoder_inputs]+decoder_state_inputs,
    [decoder_outputs]+decoder_state)

In [35]:
reverse_input_char_index=dict(
    (i,char) for char,i in input_token_index.items())

In [38]:
reverse_target_char_index=dict(
    (i,char) for char,i in target_token_index.items())

In [42]:
def decode_sequence(input_seq):
    states_value=encoder_model.predict(input_seq)
    target_seq=np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,target_token_index['\t']]=1.
    stop_condition=False
    decoded_sequence=''
    while not stop_condition:
        output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
        sampled_token_index=np.argmax(output_tokens[0,-1,:])
        sampled_char=reverse_target_char_index[sampled_token_index]
        decoded_sequence+=sampled_char
        if(sampled_char=='\n' or len(decoded_sequence)>max_decoder_seq_length):
            stop_condition=True
        target_seq=np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sampled_token_index]=1.
        states_value=[h,c]
    return decoded_sequence

In [44]:
for seq_index in range(100):
    input_seq=encoder_input_data[seq_index:seq_index+1]
    decoded_sentence=decode_sequence(input_seq)
    print('-')
    print('Input sentence:',input_texts[seq_index])
    print('Decoded sentence:',decoded_sentence)
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


KeyError: 220